In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv('tennis.csv').drop(['day'], axis = 1)
df

,outlook,temp,humidity,wind,play
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [9]:
df.sort_values(['temp'])

,outlook,temp,humidity,wind,play
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
8,Sunny,Cool,Normal,Weak,Yes
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
12,Overcast,Hot,Normal,Weak,Yes
3,Rain,Mild,High,Weak,Yes
7,Sunny,Mild,High,Weak,No


# Important formulas to note:
### $Entropy = H_{D0}(y)= -\sum_{i=0}^k p(yi)log_{2}p(yi)$
### $Weighted\ entropy = \sum_{i=0}^k \frac{|Di|}{|D|} * H_{Di}$
where; k = no. of categories in yi
### $Information\ gain = Parent\ entropy - Weighted\ entropy$
### $Gini\ impurity = 1 -\sum_{i=0}^k (p(yi))^2$


# How decision tree is created out of dataset
1. Compute initial parent entropy i.e., entropy of yi without any split
2. Compute weighted entropy for each feature
3. compute Information gain for each feature
4. Select a feature whichever gives highest Information gain
5. Now, further split as follow: 
    1. 'selected_features' will have some categories in it, sort dataset on each category.
    2. now consider sorted dataset as parent data and follow first 4 steps again
    3. whichever feature gives highest Information gain select it for further split
        4. keep doing all above steps for each category in 'selected_feature'.
6. keep doing till further split is not possible

# Information gain calculation for each feature with simulation

In [326]:
feat = 'outlook'
target = 'play'
df1 = df.sort_values([feat])
cats = df1[feat].unique()
classes = df1[target].unique()
n = len(df)
entropy_lst = []
df2_len_lst = []    # to compute weighted entropy we need to store |Di| i.e., size of each Di

# 1. initial Parent entropy 
parent_entropy = 0
for i in range(len(classes)):
    proba = df[target].value_counts()[i]/n
    parent_entropy -= (proba)*np.log2(proba)

# 2. sorting feature by each category
for i in range(len(cats)):
    proba_lst = []
    df2 = df1[df1[feat] == cats[i]]
    print(df2)
    print('-'*40)
    entropy = 0
    # 3. computing probabilities for each (Di) for each class (yi)
    for k in range(len(classes)):
        try:
            df2_len = len(df2)
            proba_lst.append(df2[target].value_counts()[k]/len(df2))
        except:
            proba_lst.append(0)
        print('p(yi={}):'.format(classes[k]), proba_lst[k])
        
    # 4. Computing child entropy for each (Di)
        if proba_lst[k] != 0:
            entropy -= proba_lst[k]*np.log2(proba_lst[k]) 
            
    df2_len_lst.append(len(df2))
    entropy_lst.append(np.round(entropy, 5))
    print('entropy:', np.round(entropy, 5))
    print("="*50)
print('Parent_entropy:', np.round(parent_entropy, 5))
# 4. Weighted Entropy 
weighted_entropy = 0
for i in range(len(entropy_lst)):
    weighted_entropy += np.round((df2_len_lst[i]/n) * entropy_lst[i],5)
print('Weighted entropy:', weighted_entropy)

# 5. Information Gain = (parent_entropy - child_weighted_entropy)
IG = np.round(parent_entropy - weighted_entropy, 5)
print('Information gain:', IG)

     outlook  temp humidity    wind play
2   Overcast   Hot     High    Weak  Yes
6   Overcast  Cool   Normal  Strong  Yes
11  Overcast  Mild     High  Strong  Yes
12  Overcast   Hot   Normal    Weak  Yes
----------------------------------------
p(yi=Yes): 1.0
p(yi=No): 0
entropy: 0.0
   outlook  temp humidity    wind play
3     Rain  Mild     High    Weak  Yes
4     Rain  Cool   Normal    Weak  Yes
5     Rain  Cool   Normal  Strong   No
9     Rain  Mild   Normal    Weak  Yes
13    Rain  Mild     High  Strong   No
----------------------------------------
p(yi=Yes): 0.6
p(yi=No): 0.4
entropy: 0.97095
   outlook  temp humidity    wind play
0    Sunny   Hot     High    Weak   No
1    Sunny   Hot     High  Strong   No
7    Sunny  Mild     High    Weak   No
8    Sunny  Cool   Normal    Weak  Yes
10   Sunny  Mild   Normal  Strong  Yes
----------------------------------------
p(yi=Yes): 0.6
p(yi=No): 0.4
entropy: 0.97095
Parent_entropy: 0.94029
Weighted entropy: 0.69354
Information gain: 0.24

In [327]:
entropy_lst

[0.0, 0.97095, 0.97095]

# Information gain calculation for each feature without simulation

In [555]:
target = 'play'
classes = df1[target].unique()
n = len(df)
IG_lst = []
# 1. initial Parent entropy 
parent_entropy = 0
entropy_lst = []
entropy_dict_lst = []
for i in range(len(classes)):
    proba = df[target].value_counts()[i]/n
    parent_entropy -= (proba)*np.log2(proba)
print('Parent_entropy:', parent_entropy)

features = df.columns[:-1]
for j in features:
    feat = j
    df1 = df.sort_values([feat])
    cats = df1[feat].unique()
    entropy_lst = []
    df2_len_lst = []    # to compute weighted entropy we need to store |Di| i.e., size of each Di
    
    # 2. sorting feature by each category
    for i in range(len(cats)):
        proba_lst = []
        df2 = df1[df1[feat] == cats[i]]
        entropy = 0
        
        # 3. computing probabilities for each (Di) for each class (yi)
        for k in range(len(classes)):
            try:
                df2_len = len(df2)
                proba_lst.append(df2[target].value_counts()[k]/len(df2))
            except:
                proba_lst.append(0)

        # 4. Computing child entropy for each (Di)
            if proba_lst[k] != 0:
                entropy -= proba_lst[k]*np.log2(proba_lst[k]) 
            
        df2_len_lst.append(len(df2))
        entropy_lst.append(np.round(entropy, 5))
        entropy_dict = dict(zip(cats, entropy_lst))
        
    entropy_dict_lst.append(entropy_dict)
#     lst.append(entropy_dict)
    # 5. Weighted Entropy 
    weighted_entropy = 0
    for i in range(len(entropy_lst)):
        weighted_entropy += np.round((df2_len_lst[i]/n) * entropy_lst[i],5)

    # 6. Information Gain = (parent_entropy - child_weighted_entropy)
    IG = np.round(parent_entropy - weighted_entropy, 5)
    print('Information gain({}):'.format(j), IG)
    IG_lst.append(IG)
IG_dict =dict(zip(features, IG_lst)) # storing IG values for further splitting
entropy_dict_lst = dict(zip(features, entropy_dict_lst))

Parent_entropy: 0.9402859586706311
Information gain(outlook): 0.24675
Information gain(temp): 0.02923
Information gain(humidity): 0.15183
Information gain(wind): 0.04813


In [556]:
IG_dict

{'outlook': 0.24675, 'temp': 0.02923, 'humidity': 0.15183, 'wind': 0.04813}

In [558]:
entropy_dict_lst

{'outlook': {'Overcast': 0.0, 'Rain': 0.97095, 'Sunny': 0.97095},
 'temp': {'Cool': 0.81128, 'Hot': 1.0, 'Mild': 0.9183},
 'humidity': {'High': 0.98523, 'Normal': 0.59167},
 'wind': {'Strong': 1.0, 'Weak': 0.81128}}

# Further splitting

In [559]:
# https://www.geeksforgeeks.org/python-get-key-with-maximum-value-in-dictionary/
max(IG_dict, key=IG_dict.get) 

'outlook'

In [584]:
feat = max(IG_dict, key=IG_dict.get) 
target = 'play'
df1 = df.sort_values([feat])
cats = df1[feat].unique()
classes = df1[target].unique()
n = len(df)
entropy_lst = []
df3_len_lst = []    # to compute weighted entropy we need to store |Di| i.e., size of each Di
features = list(df.columns[:-1])
features.remove(feat)
IG_lst = []
IG_dict_lst = []

# 2. sorting feature by each category
for i in range(len(cats)):
    if entropy_dict_lst[feat][cats[i]] != 0:
        proba_lst = []
        IG_lst = []
        df2 = df1[df1[feat] == cats[i]]
        print(df2)
        print('-'*40)
        
        # compute parent entropy
        parent_entropy = 0
        for c in range(len(classes)):
            proba = df2[target].value_counts()[c]/n
            parent_entropy -= (proba)*np.log2(proba)
        
        # 3. computing Entropy 
        for j in features:
            
            print('\nSplitting dataset on:',j)
#             entropy = 0
#             print(df2.sort_values(j))

            # 2. sorting feature by each category
            cats1 = df2[j].unique()
            entropy_lst = []
            df3_len_lst = []
            for i in range(len(cats1)):
                entropy = 0
                if entropy_dict_lst[j][cats1[i]] != 0:
                    proba_lst = []
                    df3 = df2[df2[j] == cats1[i]]
                    print(df3)
                    print('-'*40)
            
                # 4. computing probabilities for each (Di) for each class (yi)
                for k in range(len(classes)):
                    try:
                        proba_lst.append(dict(df3[target].value_counts())[classes[k]]/len(df3))
                    except:
                        proba_lst.append(0)
                    print('p(yi={}):'.format(classes[k]), proba_lst[k])
                
                    # 5. Computing child entropy for each (Di)
                    if proba_lst[k] != 0:
                        entropy -= proba_lst[k]*np.log2(proba_lst[k]) 
                df3_len_lst.append(len(df3))
                entropy_lst.append(np.round(entropy, 5))
                entropy_dict = dict(zip(cats, entropy_lst))
                print('entropy:', np.round(entropy, 5))
                print("="*40)
            print('*'*50)
            print('Parent_entropy({}):'.format(cats1[i]), np.round(parent_entropy, 5))
                
            # 4. Weighted Entropy 
            weighted_entropy = 0
            for e in range(len(entropy_lst)):
                weighted_entropy += np.round((df3_len_lst[e]/len(df2)) * entropy_lst[e],5)
            print('Weighted entropy({}):'.format(j), weighted_entropy)

            # 5. Information Gain = (parent_entropy - child_weighted_entropy)
            IG = np.round(parent_entropy - weighted_entropy, 5)
            print('Information gain({}):'.format(j), IG)
            print('*'*50)
            IG_lst.append(IG)
    IG_dict_lst.append(dict(zip(features, IG_lst)))
IG_final_dict = dict(zip(cats, IG_dict_lst))

   outlook  temp humidity    wind play
3     Rain  Mild     High    Weak  Yes
4     Rain  Cool   Normal    Weak  Yes
5     Rain  Cool   Normal  Strong   No
9     Rain  Mild   Normal    Weak  Yes
13    Rain  Mild     High  Strong   No
----------------------------------------

Splitting dataset on: temp
   outlook  temp humidity    wind play
3     Rain  Mild     High    Weak  Yes
9     Rain  Mild   Normal    Weak  Yes
13    Rain  Mild     High  Strong   No
----------------------------------------
p(yi=Yes): 0.6666666666666666
p(yi=No): 0.3333333333333333
entropy: 0.9183
  outlook  temp humidity    wind play
4    Rain  Cool   Normal    Weak  Yes
5    Rain  Cool   Normal  Strong   No
----------------------------------------
p(yi=Yes): 0.5
p(yi=No): 0.5
entropy: 1.0
**************************************************
Parent_entropy(Cool): 0.87728
Weighted entropy(temp): 0.95098
Information gain(temp): -0.0737
**************************************************

Splitting dataset on: humidity


In [585]:
IG_final_dict

{'Overcast': {},
 'Rain': {'temp': -0.0737, 'humidity': -0.0737, 'wind': 0.87728},
 'Sunny': {'temp': 0.47728, 'humidity': 0.87728, 'wind': -0.0737}}